<a href="https://colab.research.google.com/github/Rohan5076/AIMLOPS/blob/main/M6_MiniProject2_RAG_with_Open_Source_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [ ]:
%%capture
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-chroma
!pip -q install chromadb
!pip -q install pypdf

Importing Required Packages

In [ ]:
import os
import numpy as np
from getpass import getpass
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

Authenticating Huggingface API

In [ ]:
import os
from getpass import getpass

hfapi_key = getpass("Enter you HuggingFace access token:")
os.environ["HF_TOKEN"] = hfapi_key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfapi_key

Enter you HuggingFace access token:··········


Accessing Open Source LLM

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens = 512,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading the documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf"),
    PyPDFLoader("/content/Sebastian Raschka - Build a Large Language Model (From Scratch)-Manning Publications Co. (2024).pdf"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [ ]:
len(docs)

814

In [ ]:
docs

[Document(metadata={'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf', 'page': 1}, page_content='W h a t  E x p e r t s  T h i n k  A b o u t  B u i l d i n g  L L M s  f o r\nP r o d u c t i o n\n" T h i s  i s  t h e  m o s t  c o m p r e h e n s i v e  t e x t b o o k  t o  d a t e  o n  b u i l d i n g  L L M\na p p l i c a t i o n s ,  a n d  h e l p s  l e a r n e r s  u n d e r s t a n d  e v e r y t h i n g  f r o m\nf u n d a m e n t a l s  t o  t h e  s i m p l e - t o - a d v a n c e d  b u i l d i n g  b l o c k s  o f\nc o n s t r u c t i n g  L L M  a p p l i c a t i o n s .  T h e  a p p l i c a t i o n  t o p i c s  i n c l u d e\np r o m p t i n g ,  R A G ,  a g e n t s ,  ﬁ n e - t u n i n g ,  a n d  d e p l o y m e n t  -  a l l  e s s e n t i a l\nt o p i c s  i n  a n  A I  E n g i n

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)

print(len(splits))
print(len(splits[0].page_content) )
splits[0].page_content

3126
435


'W h a t  E x p e r t s  T h i n k  A b o u t  B u i l d i n g  L L M s  f o r\nP r o d u c t i o n\n" T h i s  i s  t h e  m o s t  c o m p r e h e n s i v e  t e x t b o o k  t o  d a t e  o n  b u i l d i n g  L L M\na p p l i c a t i o n s ,  a n d  h e l p s  l e a r n e r s  u n d e r s t a n d  e v e r y t h i n g  f r o m\nf u n d a m e n t a l s  t o  t h e  s i m p l e - t o - a d v a n c e d  b u i l d i n g  b l o c k s  o f'

In [ ]:
splits[0]

Document(metadata={'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf', 'page': 1}, page_content='W h a t  E x p e r t s  T h i n k  A b o u t  B u i l d i n g  L L M s  f o r\nP r o d u c t i o n\n" T h i s  i s  t h e  m o s t  c o m p r e h e n s i v e  t e x t b o o k  t o  d a t e  o n  b u i l d i n g  L L M\na p p l i c a t i o n s ,  a n d  h e l p s  l e a r n e r s  u n d e r s t a n d  e v e r y t h i n g  f r o m\nf u n d a m e n t a l s  t o  t h e  s i m p l e - t o - a d v a n c e d  b u i l d i n g  b l o c k s  o f')

Embedding the splits

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [ ]:
# Embedding Model

from langchain_huggingface import HuggingFaceEmbeddings

modelPath ="mixedbread-ai/mxbai-embed-large-v1"                  # Model card: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': device}      # cuda/cpu

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

embedding =  HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

Creating Vector stores

In [ ]:
from langchain_chroma import Chroma       # Light-weight and in memory

In [ ]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,                    # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory, # save the directory
)

In [ ]:
print(vectordb._collection.count()) # same as number of splits

3126


In [ ]:
question = "What is RAG? Explain in 5 sentences."

In [ ]:
docs = vectordb.similarity_search(question, k=6)     # k --> No. of Document object to return

In [ ]:
print(len(docs))

for i in range(len(docs)):
    print(docs[i].page_content)
    print('='*140)

# print(docs[2].metadata['page'])

6
answer from the context paragraphs and not write anything that
cannot be inferred from them.
Retrieval-augmented generation (RAG) is a technique for enhancing the
capabilities of language models by adding data from external sources.
This information is combined with the context already included in the
model’s prompt, allowing the model to offer more accurate and relevant
responses.
Access to external data sources during the generation phase significantly
Chapter VIII: Advanced RAG
follow complex instructions. However, it requires a large, high-quality
dataset labeled for a specific task. RAG is designed to integrate external
knowledge, allowing the model to access a wide range of up-to-date and
diverse information. However, RAG is more complex to integrate and
resource-intensive.
Integrating components such as query expansion, transformations, and
construction techniques leads to the creation of an efficient retrieval engine,
instrumental in assessing the capabilities of an RAG syste

Retrieval - Vectorstore as a retriever

In [ ]:
question = "What is RAG? Explain in 5 sentences."
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k":5})
docs = retriever.invoke(question)
docs

[Document(metadata={'page': 108, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='answer from the context paragraphs and not write anything that\ncannot be inferred from them.\nRetrieval-augmented generation (RAG) is a technique for enhancing the\ncapabilities of language models by adding data from external sources.\nThis information is combined with the context already included in the\nmodel’s prompt, allowing the model to offer more accurate and relevant\nresponses.\nAccess to external data sources during the generation phase significantly'),
 Document(metadata={'page': 311, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='Chapter VIII: Advanced RAG')]

Augmentation



In [ ]:
from langchain_core.prompts import PromptTemplate                                    # To format prompts
from langchain_core.output_parsers import StrOutputParser                            # to transform the output of an LLM into a more usable format
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough          # Required by LCEL (LangChain Expression Language)

In [ ]:
# Build prompt template
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

Generation - RAG Chain

In [ ]:
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 7, "fetch_k":15})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7c0d36d51360>, search_type='mmr', search_kwargs={'k': 7, 'fetch_k': 15})

In [ ]:
question = "What is RAG? Explain in 5 sentences."
docs = retriever.invoke(question)
docs

[Document(metadata={'page': 108, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='answer from the context paragraphs and not write anything that\ncannot be inferred from them.\nRetrieval-augmented generation (RAG) is a technique for enhancing the\ncapabilities of language models by adding data from external sources.\nThis information is combined with the context already included in the\nmodel’s prompt, allowing the model to offer more accurate and relevant\nresponses.\nAccess to external data sources during the generation phase significantly'),
 Document(metadata={'page': 311, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='Chapter VIII: Advanced RAG'),
 Document(metadata={'page': 348, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='instrumental in assessing the capabilities of an RAG system in terms of\nquestion

In [ ]:
retrieval = RunnableParallel(
    {
        "context": RunnablePassthrough(context= lambda x: x["question"] | retriever),
        "question": RunnablePassthrough()
        }
    )

In [ ]:
# RAG Chain

rag_chain = (retrieval                     # Retrieval
             | QA_PROMPT                   # Augmentation
             | llm                         # Generation
             | StrOutputParser()
             )

Checking what is being retreived from the retriever

In [ ]:
chain_retriever = RunnablePassthrough() | retriever

In [ ]:
chain_retriever.invoke("What is Rag?")

[Document(metadata={'page': 311, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='Chapter VIII: Advanced RAG'),
 Document(metadata={'page': 108, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='answer from the context paragraphs and not write anything that\ncannot be inferred from them.\nRetrieval-augmented generation (RAG) is a technique for enhancing the\ncapabilities of language models by adding data from external sources.\nThis information is combined with the context already included in the\nmodel’s prompt, allowing the model to offer more accurate and relevant\nresponses.\nAccess to external data sources during the generation phase significantly'),
 Document(metadata={'page': 360, 'source': '/content/Building LLMs for Production Louis François Bouchard Louie Peters.pdf'}, page_content='follow complex instructions. However, it requires a large, high-quality\ndataset 

Executing the entire RAG Chain

In [ ]:
response = rag_chain.invoke({"question": "What is RAG?"})

response

' RAG is a system used to categorize the status of tasks or projects based on their risk, urgency, and progress. It uses three colors: Red (high risk/urgency), Amber (medium risk/urgency), and Green (low risk/urgency). Thanks for asking!'

In [ ]:
response = rag_chain.invoke({"question": "What is principal component analysis?"})

response

' Principal Component Analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The first principal component has the largest possible variance, and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. PCA is sensitive to the relative scaling of the original variables, so it is important to standardize the data before performing PCA. Thanks for asking!'

In [ ]:
response = rag_chain.invoke({"question": "How ensemble method works?"})

print(response)


Ensemble methods are a type of machine learning technique that combines multiple models to improve the overall performance and accuracy. The basic idea is to train multiple models on the same data, and then combine their predictions to make a final prediction.

There are several types of ensemble methods, but the most common ones are Bagging, Boosting, and Stacking.

1. Bagging (Bootstrap Aggregating): This method trains multiple models on different subsets of the training data, each subset being a random sample with replacement. The final prediction is made by taking the average or majority vote of the individual predictions.

2. Boosting: This method trains multiple models sequentially, with each model focusing on the instances that were misclassified by the previous models. The final prediction is made by combining the predictions of all the models, with more weight given to the predictions of the later models.

3. Stacking: This method involves training multiple models, and then u

In [ ]:
# For queries that is not in documents
response = rag_chain.invoke({"question": "Tell me what you don't know."})

print(response)           # It should return "I don't know. Thanks for asking!". The open-source model used is not that great.

 I don't know what I don't know. Thanks for asking!


Creating a Gradio Appto write query and get response from the RAG System

In [ ]:
!pip install gradio
import gradio as gr

def generate_text(prompt):
    return rag_chain.invoke(prompt)

iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Enter your prompt", placeholder="Write your prompt here"),
    ],
    outputs=[
        gr.Textbox(label="Generated Text"),
    ],
    live=True,
)

iface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3be8ac2a2ba7e67d91.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
